# Config

In [1]:
NUMBER_OF_IMAGES=600

In [2]:
OPENAI_API_KEY=""
HUGGINGFACE_API_KEY=""

# Prompt

In [28]:
role_identifier_prompt = """
You are part of a team tasked with generating role-aware and context-aware image alt texts for images on websites. Your role is to identify the role of the given image in the website according to the definitions provided by the WCAG Web Accessibility Initiative (WAI) outlined below.


- informative: Images that graphically represent concepts and information, typically pictures, photos, and illustrations. The text alternative should be at least a short description conveying the essential information presented by the image.

- decorative: The only purpose of an image is to add visual decoration to the page, rather than to convey information that is important to understanding the page. This includes images that are considered eye candy or used for visual effect. Classify the image to decorative if having a null alt-text (alt="") will not result in any loss of information.

- functional: Images used as a link or as a button, which carry a functionality to the page. Examples of such images are a printer icon to represent the print function or a button to submit a form. The alt text should describe the functionality of the link or button rather than the visual image.

- complex: Images used to convey data or detailed information, such as graphs or charts. Alt texts provide a complete text equivalent of the data or information provided in the image as the text alternative.


As each role needs to be handled differently when generating alt texts, your output will be used to help another team member write the most suitable alt text that is role-aware and contex-aware for the image to help create more accessible websites.

Return only the role of the image from the list above. Return the role as a single word without any enclosing bracket or other punctuations (informative, decorative, functional, text, or complex). THIS IS IMPORTANT! RETURN ONLY THE ROLE OF THE IMAGE.

Here is the detail about the image: 
{message}
"""

In [4]:
# One-shot
role_identifier_prompt = """
You are part of a team tasked with generating role-aware and context-aware image alt texts for images on websites. Your role is to identify the role of the given image in the website according to the definitions provided by the WCAG Web Accessibility Initiative (WAI) outlined below.


- informative: Images that graphically represent concepts and information, typically pictures, photos, and illustrations. The text alternative should be at least a short description conveying the essential information presented by the image.

- decorative: The only purpose of an image is to add visual decoration to the page, rather than to convey information that is important to understanding the page. This includes images that are considered eye candy or used for visual effect. Classify the image to decorative if having a null alt-text (alt="") will not result in any loss of information.

- functional: Images used as a link or as a button, which carry a functionality to the page. Examples of such images are a printer icon to represent the print function or a button to submit a form. The alt text should describe the functionality of the link or button rather than the visual image.

- complex: Images used to convey data or detailed information, such as graphs or charts. Alt texts provide a complete text equivalent of the data or information provided in the image as the text alternative.


Example:
You are given an image of a dog wearing a bell, with the following context:
The image's attributes: {
      "class": ["img"],
      "id": "dog-off-duty",
      "src": "",
      "alt": ""
    }
The image's <a> or <button> parent: None
The previous text before the image appears: None
The next text after the image appears: Off-duty guide dogs often wear a bell. Its ring helps the blind owner keep track of the dog's location

In this case, you should output "informative" as the image is a picture of a dog wearing a bell, which is essential information for the reader to understand the context of the text.


As each role needs to be handled differently when generating alt texts, your output will be used to help another team member write the most suitable alt text that is role-aware and contex-aware for the image to help create more accessible websites.

Return only the role of the image from the list above. Return the role as a single word without any enclosing bracket or other punctuations (informative, decorative, functional, text, or complex). THIS IS IMPORTANT! RETURN ONLY THE ROLE OF THE IMAGE.
"""

In [16]:
# Few-shot
role_identifier_prompt = """
You are part of a team tasked with generating role-aware and context-aware image alt texts for images on websites. Your role is to identify the role of the given image in the website according to the definitions provided by the WCAG Web Accessibility Initiative (WAI) outlined below.


- informative: Images that graphically represent concepts and information, typically pictures, photos, and illustrations. The text alternative should be at least a short description conveying the essential information presented by the image.

- decorative: The only purpose of an image is to add visual decoration to the page, rather than to convey information that is important to understanding the page. This includes images that are considered eye candy or used for visual effect. Classify the image to decorative if having a null alt-text (alt="") will not result in any loss of information.

- functional: Images used as a link or as a button, which carry a functionality to the page. Examples of such images are a printer icon to represent the print function or a button to submit a form. The alt text should describe the functionality of the link or button rather than the visual image.

- complex: Images used to convey data or detailed information, such as graphs or charts. Alt texts provide a complete text equivalent of the data or information provided in the image as the text alternative.


Example 1:
You are given an image of a dog wearing a bell, with the following context:
The image's attributes: {
    "class": ["img"],
    "id": "dog-off-duty",
    "src": "",
    "alt": ""
}
The image's <a> or <button> parent: None
The previous text before the image appears: None
The next text after the image appears: Off-duty guide dogs often wear a bell. Its ring helps the blind owner keep track of the dog's location

In this case, you should output "informative" as the image is a picture of a dog wearing a bell, which is essential information for the reader to understand the context of the text.

Example 2:
You are given a logo image of RealPage, Inc., with the following context:
The image's attributes: {
    "src": "/TemplateResources/Global/images/rplogo-white.png",
    "alt": "RealPage, Inc."
}
The image's <a> or <button> parent: "<a href=\"https://www.realpage.com/apartment-marketing/\" target=\"_blank\"><img alt=\"RealPage, Inc.\" src=\"/TemplateResources/Global/images/rplogo-white.png\"/></a>"
The previous text before the image appears: address
The next text after the image appears: Privacy Preference Center

In this case, you should output "functional" as the image is a logo used as a link to the RealPage, Inc. website, apparent from the parent <a> tag. If there is no parent <a> or <button> tag, the image would be considered "informative" if it provides essential information or "decorative" if it is used for visual decoration.


Example 3:
You are given a stock photo of a busy conference check in desk, with the following context:
The image's attributes: {
    "class": ["card-img-top"],
}
The image's <a> or <button> parent: None
The previous text before the image appears: Print delegate badge instantly with fast and delightful experience.
The next text after the image appears: Smart Mobile Check-in

In this case, you should output "decorative" as the image is used for visual decoration and does not convey any essential information, as the previous and next text already describe the relevant context.


Example 4:
You are given multiple screenshots of a product's user interface in different devices, with the following context:
The image's attributes: {
    "width": "1024",
    "height": "614",
    "class": [
        "attachment-large",
        "size-large",
        "wp-image-37835"
    ],
    "sizes": "(max-width: 1024px) 100vw, 1024px"
},
"input_a_button_parent": "None",
"input_next_text": "Our Company",

In this case, you should output "complex" as the image is used to convey detailed information about the product's user interface.


As each role needs to be handled differently when generating alt texts, your output will be used to help another team member write the most suitable alt text that is role-aware and contex-aware for the image to help create more accessible websites.

Return only the role of the image from the list above. Return the role as a single word without any enclosing bracket or other punctuations (informative, decorative, functional, text, or complex). THIS IS IMPORTANT! RETURN ONLY THE ROLE OF THE IMAGE.
"""

In [9]:
role_identifier_prompt = """
You are part of a team tasked with generating role-aware and context-aware image alt texts for images on websites. Your role is to identify the role of the given image in the website according to the definitions provided by the WCAG Web Accessibility Initiative (WAI). Classify the image into "informative", "decorative", "functional", or "complex". Return the role as a single word without any enclosing bracket or other punctuations (informative, decorative, functional, text, or complex). THIS IS IMPORTANT! RETURN ONLY THE ROLE OF THE IMAGE.
"""

# Load Data

In [4]:
import json
import os
import random

json_dir = "../../scraper/output"

filenames = os.listdir(json_dir)

# Shuffle the filenames
random.seed(42)
random.shuffle(filenames)

# Llama-3.2-11B-Vision-Instruct

In [30]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

def determine_role_llama(image):
    image_details = f"""
        The image's attributes: {json.dumps(image["attrs"])}\n\n
        The image's <a> or <button> parent: {image["a_button_parent"]}\n\n
        The previous text before the image appears: {image["previous_text"]}\n\n
        The next text after the image appears: {image["next_text"]}\n\n
    """

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image["src"]
                    }
                },
                {
                    "type": "text",
                    "text": role_identifier_prompt.format(message=image_details)
                },
            ]
        }
    ]

    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.2-11B-Vision-Instruct",
        messages=messages,
        max_tokens=500
    )

    return completion.choices[0].message.content

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

def llama_test():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Explain about the 7 image roles defined by WCAG (decorative, informative, functional, complex, text, images of maps, and group images)."
                },
            ]
        }
    ]

    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.2-11B-Vision-Instruct",
        messages=messages,
        max_tokens=500
    )

    return completion.choices[0].message.content

print(llama_test())

The World Wide Web Consortium's (W3C) Web Content Accessibility Guidelines (WCAG) 2.1 defines seven image roles to help developers and designers create more accessible web content. These roles are essential for ensuring that all users, including those with visual impairments, can understand and interact with visual content on the web. Here are the seven image roles defined by WCAG:

1. **Decorative**: Decorative images are intended to be purely aesthetic and do not provide any information or function. They may be used to add visual interest or style to a website, but they do not enhance the content or provide any assistive information. Decorative images should be hidden or removed from the accessibility tree to prevent them from distracting from the main content. (Example: A patterned background image)

2. **Informative**: Informative images provide information that is essential to understanding the content, but the information is not solely contained within the image itself. These ima

# SmolVLM

In [8]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load images
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")

# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16,
).to(DEVICE)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Can you describe the two images?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])
"""
Assistant: The first image shows a green statue of the Statue of Liberty standing on a stone pedestal in front of a body of water. 
The statue is holding a torch in its right hand and a tablet in its left hand. The water is calm and there are no boats or other objects visible. 
The sky is clear and there are no clouds. The second image shows a bee on a pink flower. 
The bee is black and yellow and is collecting pollen from the flower. The flower is surrounded by green leaves.
"""


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/felinejtd/anaconda3/envs/thesis/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/felinejtd/anaconda3/envs/thesis/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/felinejtd/anaconda3/envs/thesis/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

ValueError: The checkpoint you are trying to load has model type `idefics3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

# Qwen-7B

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

def determine_role_qwen_7b(image):
    image_details = f"""
        The image's attributes: {json.dumps(image["attrs"])}\n\n
        The image's <a> or <button> parent: {image["a_button_parent"]}\n\n
        The previous text before the image appears: {image["previous_text"]}\n\n
        The next text after the image appears: {image["next_text"]}\n\n
    """

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image["src"]
                    }
                },
                {
                    "type": "text",
                    "text": role_identifier_prompt.format(message=image_details)
                },
            ]
        }
    ]

    completion = client.chat.completions.create(
        model="Qwen/Qwen2-VL-7B-Instruct",
        messages=messages,
        max_tokens=500
    )

    return completion.choices[0].message.content

In [24]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

def qwen_test():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Explain about the 7 image roles defined by WCAG (decorative, informative, functional, complex, text, images of maps, and group images)."
                },
            ]
        }
    ]

    completion = client.chat.completions.create(
        model="Qwen/Qwen2-VL-7B-Instruct",
        messages=messages,
        max_tokens=500
    )

    return completion.choices[0].message.content

print(qwen_test())

The Web Content Accessibility Guidelines现在很多都用7个imag


# Qwen-2B

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

def determine_role_qwen_2b(image):
    image_details = f"""
        The image's attributes: {json.dumps(image["attrs"])}\n\n
        The image's <a> or <button> parent: {image["a_button_parent"]}\n\n
        The previous text before the image appears: {image["previous_text"]}\n\n
        The next text after the image appears: {image["next_text"]}\n\n
    """

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image["src"]
                    }
                },
                {
                    "type": "text",
                    "text": role_identifier_prompt.format(message=image_details)
                },
            ]
        }
    ]

    completion = client.chat.completions.create(
        model="Qwen/Qwen2-VL-2B-Instruct",
        messages=messages,
        max_tokens=500
    )

    return completion.choices[0].message.content

/home/felinejtd/anaconda3/envs/thesis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)

def qwen_test():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Explain about the 7 image roles defined by WCAG (decorative, informative, functional, complex, text, images of maps, and group images)."
                },
            ]
        }
    ]

    completion = client.chat.completions.create(
        model="Qwen/Qwen2-VL-2B-Instruct",
        messages=messages,
        max_tokens=500
    )

    return completion.choices[0].message.content

print(qwen_test())

Web Content Accessibility Guidelines or WCAG is a set of recommended practices to ensure that web content is accessible to people with a variety of disabilities including people with physical and visual impairments/access, and people who use assistive technology like screen readers/accessories/additional  language texts in other larger text, and text on demand and data control. The 7 roles of the  are of help with the  the of building a  and to  people with  and and  and  and to ensure that information can be easily and  for people with  and to multiple  and to and when  and . Interpret what each of the and how to of and and  any to and any to build on to and to for and  for and to and to the and design and  and and for the and of and and and and and and and and of and and and and of and and and and and and and and and and and and each when and the and of and and and and for the and of the for of and of for the in and and to and and for the and of and of and is for and to and and and t

# GPT-4o

In [5]:
# !pip install langchain_openai

from langchain_openai import ChatOpenAI

def determine_role_gpt_4o(image):
    image_details = f"""
        The image's attributes: {json.dumps(image["attrs"])}\n\n
        The image's <a> or <button> parent: {image["a_button_parent"]}\n\n
        The previous text before the image appears: {image["previous_text"]}\n\n
        The next text after the image appears: {image["next_text"]}\n\n
    """

    role_identifier_llm = ChatOpenAI(model='ft:gpt-4o-2024-08-06:personal:role-iden-50:AahYYJoD', temperature=0.5, api_key=OPENAI_API_KEY)
    predicted_role = role_identifier_llm.invoke(
        [
            (
                "system",
                role_identifier_prompt
            ),
            (
                "human",
                [
                    {
                        "type": "image_url", "image_url": {"url": image["src"]}
                    },
                    {
                        "type": "text", "text": image_details
                    }
                ]
            )
        ]
    )

    return predicted_role.content

In [12]:
# !pip install langchain_openai

from langchain_openai import ChatOpenAI

def gpt_4o_test():
    role_identifier_llm = ChatOpenAI(model='gpt-4o', temperature=0.5, api_key=OPENAI_API_KEY)
    predicted_role = role_identifier_llm.invoke(
        [
            (
                "human",
                [
                    {
                        "type": "text", "text": "Tell me the 7 image roles defined by WCAG."
                    }
                ]
            )
        ]
    )

    return predicted_role.content

print(gpt_4o_test())

The Web Content Accessibility Guidelines (WCAG) do not explicitly define "image roles" per se, but they do provide guidance on how to make images accessible. This guidance can be interpreted in terms of different roles or purposes that images might serve on a webpage. Here are seven common roles or purposes for images in the context of web accessibility, along with how they should be handled according to WCAG:

1. **Informative Images**: These images convey simple information or concepts that can be expressed in a short phrase or sentence. They should have descriptive alternative text (alt text) that conveys the same information as the image.

2. **Decorative Images**: These images are used for aesthetic purposes and do not convey any meaningful information. They should have empty alt text (`alt=""`) so that screen readers can skip them.

3. **Functional Images**: These images are used as buttons or links and have a functional purpose. The alt text should describe the function or actio

# GPT-4o-mini

In [7]:
from langchain_openai import ChatOpenAI

def determine_role_gpt_4o_mini(image):
    image_details = f"""
        The image's attributes: {json.dumps(image["attrs"])}\n\n
        The image's <a> or <button> parent: {image["a_button_parent"]}\n\n
        The previous text before the image appears: {image["previous_text"]}\n\n
        The next text after the image appears: {image["next_text"]}\n\n
    """

    role_identifier_llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.5, api_key=OPENAI_API_KEY)
    predicted_role = role_identifier_llm.invoke(
        [
            (
                "system",
                role_identifier_prompt
            ),
            (
                "human",
                [
                    {
                        "type": "image_url", "image_url": {"url": image["src"]}
                    },
                    {
                        "type": "text", "text": image_details
                    }
                ]
            )
        ]
    )

    return predicted_role.content

In [ ]:
# !pip install langchain_openai

from langchain_openai import ChatOpenAI

def gpt_4o_mini_test():
    role_identifier_llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.5, api_key=OPENAI_API_KEY)
    predicted_role = role_identifier_llm.invoke(
        [
            (
                "human",
                [
                    {
                        "type": "text", "text": "Tell me the 7 image roles defined by WCAG."
                    }
                ]
            )
        ]
    )

    return predicted_role.content

print(gpt_4o_mini_test())

The Web Content Accessibility Guidelines (WCAG) define several roles for images to ensure that they are accessible to all users, including those with disabilities. The seven image roles defined by WCAG are:

1. **Decorative Images**: These images are purely for decorative purposes and do not convey any meaningful content. They should have an empty `alt` attribute (e.g., `alt=""`) so that screen readers can skip them.

2. **Informative Images**: These images convey information that is essential to understanding the content. They should have a descriptive `alt` attribute that conveys the meaning of the image.

3. **Functional Images**: These images are used as links or buttons (e.g., icons). They should have an `alt` attribute that describes the function of the image (e.g., "Search" for a magnifying glass icon).

4. **Textual Images**: These images contain text that is important for understanding the content. They should be accompanied by an `alt` attribute that describes the text or the

# Experiment

In [6]:
def calculate_scores(results):
    scores = {
        "llama": {
            "whole_accuracy": 0,
            "precision": {},
            "recall": {},
            "f1": {},
        },
        "gpt-4o": {
            "whole_accuracy": 0,
            "precision": {},
            "recall": {},
            "f1": {},
        },
        "gpt-4o-mini": {
            "whole_accuracy": 0,
            "precision": {},
            "recall": {},
            "f1": {},
        },
        "qwen": {
            "whole_accuracy": 0,
            "precision": {},
            "recall": {},
            "f1": {},
        }
    }

    for model in results["details"]:
        # Whole accuracy
        scores[model]["whole_accuracy"] = sum([results["details"][model][role]["true_positive"] for role in results["details"][model]]) / sum([results["details"][model][role]["true_positive"] + results["details"][model][role]["false_positive"] for role in results["details"][model]]) if sum([results["details"][model][role]["true_positive"] + results["details"][model][role]["false_positive"] for role in results["details"][model]]) > 0 else 0

        # Accuracy, Precision, Recall, F1 for each role
        for role in results["details"][model]:
            true_positive = results["details"][model][role]["true_positive"]
            false_positive = results["details"][model][role]["false_positive"]
            false_negative = results["details"][model][role]["false_negative"]

            scores[model]["precision"][role] = true_positive / (true_positive + false_positive) if true_positive + false_positive > 0 else 0
            scores[model]["recall"][role] = true_positive / (true_positive + false_negative) if true_positive + false_negative > 0 else 0
            scores[model]["f1"][role] = 2 * (scores[model]["precision"][role] * scores[model]["recall"][role]) / (scores[model]["precision"][role] + scores[model]["recall"][role]) if scores[model]["precision"][role] + scores[model]["recall"][role] > 0 else 0

    return scores

In [10]:
results = {
    "total_images": 0,
    "details": {
        "llama" : {
            "informative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "decorative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "functional": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "complex": {"true_positive": 0, "false_positive": 0, "false_negative": 0}
        },
        "gpt-4o" : {
            "informative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "decorative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "functional": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "complex": {"true_positive": 0, "false_positive": 0, "false_negative": 0}
        },
        "gpt-4o-mini" : {
            "informative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "decorative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "functional": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "complex": {"true_positive": 0, "false_positive": 0, "false_negative": 0}
        },
        "qwen" : {
            "informative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "decorative": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "functional": {"true_positive": 0, "false_positive": 0, "false_negative": 0},
            "complex": {"true_positive": 0, "false_positive": 0, "false_negative": 0}
        }
    }
}

total_images = 0

for filename in os.listdir(json_dir):
    if total_images >= NUMBER_OF_IMAGES:
        break

    if filename.endswith(".json") and filename != "aquapure-dicorium.com.json" and filename != "output.json":
        try:
            # Read the JSON file
            with open(os.path.join(json_dir, filename), "r") as file:
                data = json.load(file)

            print(f"Processing {filename}...")

            # Extract the image link and textual context from the JSON data
            whole_text = data["text"]
            sub_images = data["images"]

            for image in sub_images:
                try:
                    if image["role"] != "informative" and image["role"] != "decorative" and image["role"] != "functional" and image["role"] != "complex":
                        continue

                    if total_images >= NUMBER_OF_IMAGES:
                        break

                    total_images += 1
                    results["total_images"] += 1

                    print("Correct role:", image["role"])

                    # try:
                    #     llama_answer = determine_role_llama(image).strip().lower().replace(".", "")
                    #     print("Predicted role (Llama):", llama_answer)
                    #     if llama_answer == image["role"]:
                    #         results["details"]["llama"][image["role"]]["true_positive"] += 1
                    #     else:
                    #         results["details"]["llama"][image["role"]]["false_negative"] += 1
                    #         if llama_answer in results["details"]["llama"]:
                    #             results["details"]["llama"][llama_answer]["false_positive"] += 1
                    # except Exception as e:
                    #     print(str(e))


                    # try:
                    #     qwen_answer = determine_role_qwen_2b(image).strip().lower().replace(".", "")
                    #     print("Predicted role (Qwen):", qwen_answer)
                    #     if qwen_answer == image["role"]:
                    #         results["details"]["qwen"][image["role"]]["true_positive"] += 1
                    #     else:
                    #         results["details"]["qwen"][image["role"]]["false_negative"] += 1
                    #         if qwen_answer in results["details"]["qwen"]:
                    #             results["details"]["qwen"][qwen_answer]["false_positive"] += 1
                    # except Exception as e:
                    #     print(str(e))


                    try:
                        gpt_4o_answer = determine_role_gpt_4o(image).strip().lower().replace(".", "")
                        print("Predicted role (GPT-4o):", gpt_4o_answer)
                        if gpt_4o_answer == image["role"]:
                            results["details"]["gpt-4o"][image["role"]]["true_positive"] += 1
                        else:
                            results["details"]["gpt-4o"][image["role"]]["false_negative"] += 1
                            if gpt_4o_answer in results["details"]["gpt-4o"]:
                                results["details"]["gpt-4o"][gpt_4o_answer]["false_positive"] += 1
                    except Exception as e:
                        print(str(e))


                    # try:
                    #     gpt_4o_mini_answer = determine_role_gpt_4o_mini(image).strip().lower().replace(".", "")
                    #     print("Predicted role (GPT-4o Mini):", gpt_4o_mini_answer)
                    #     if gpt_4o_mini_answer == image["role"]:
                    #         results["details"]["gpt-4o-mini"][image["role"]]["true_positive"] += 1
                    #     else:
                    #         results["details"]["gpt-4o-mini"][image["role"]]["false_negative"] += 1
                    #         if gpt_4o_mini_answer in results["details"]["gpt-4o-mini"]:
                    #             results["details"]["gpt-4o-mini"][gpt_4o_mini_answer]["false_positive"] += 1
                    # except Exception as e:
                    #     print(str(e))

                except Exception as e:
                    print(str(e))

            # Save the final state to a JSON file
            with open(f"output-gpt-4o-finetuned.json", "w") as file:
                json.dump({
                    "results": results,
                    "scores": calculate_scores(results)
                }, file, indent=4)

        except Exception as e:
            print(str(e))

print("Total images processed:", results["total_images"])
print("Results:", results)
print("Scores:", calculate_scores(results))

Processing hannarubbercompany.com.json...
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GPT-4o): functional
Correct role: functional
Predicted role (GP